In [33]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from matplotlib.cm import tab20 as cmap
from matplotlib import pyplot as plt
import laspy as lp

# 1. Load train and test data

In [16]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
x_new_test2 = np.load('test_data_second.npy')

In [17]:
test_data = x_new_test2[:,3:]
test_coordinates = x_new_test2[:,:3]

# 2. Normalize the data

In [18]:
# Create a scaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform the data
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.fit_transform(test_data)

# 3. Design the NN and Random Forest

In [19]:
# random forest

In [20]:
# Define your Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=100)

# Train your Random Forest Classifier on your dataset
rfc.fit(X_train_scaled, y_train)

RandomForestClassifier()

In [21]:
# predict
rfc_probs = rfc.predict_proba(X_test_scaled)

In [29]:
rfc_pred = np.argmax(rfc_probs, axis=1)

In [22]:
# neural network

In [ ]:
# Define your Neural Network
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=8))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax')) # 7 is the number of classes in your target variable

# Compile your Neural Network
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train your Neural Network on the same validation set
history = model.fit(X_train_scaled, y_train, epochs=8, batch_size=32)

Epoch 1/8
 348/9376 [>.............................] - ETA: 39s - loss: 0.5438 - accuracy: 0.8512

KeyboardInterrupt: 

In [24]:
# Use your trained Neural Network to predict probabilities on new data
nn_probs = model.predict(X_test_scaled)

# Convert the probabilities to class labels
nn_labels = np.argmax(nn_probs, axis=-1)

5040/5040 [==============================] - 10s 2ms/step


In [25]:
# Optional testing to combine them

In [26]:
# Combine the probabilities from both models using a weighted average
ensemble_probs = 0.7 * rfc_probs + 0.3 * nn_probs

# Get the predicted class labels from the ensemble probabilities
ensemble_preds = np.argmax(ensemble_probs, axis=1)

# 4. Save the output as a new las file

In [28]:
print_coord = x_new_test2[:, :3]

In [35]:
fn = "nn_labelsrfc_pred_preds.las"

vals = np.linspace(0, 1, 100)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.tab20(vals))
header = lp.header.Header()
header.data_format_id = 2
fp = lp.file.File(fn, mode = 'w', header = header)
fp.header.scale = [0.01, 0.01, 0.01]
fp.header.offset = [min(print_coord[:,0]), min(print_coord[:,1]), min(print_coord[:,2])]
fp.x = print_coord[:, 0]
fp.y = print_coord[:, 1]
fp.z = print_coord[:, 2]
fp.pt_src_id = nn_labels
#fp.intensity = intensity
fp.close()